## Étape 1 – Profiling DataCoSupplyChainDataset.csv

### 1. Vue d’ensemble

- Lignes : **180 519**
- Colonnes : **53**
- Type de données :
  - Beaucoup de colonnes **catégorielles** (object) : statut de livraison, pays, ville, marché, mode d’expédition, etc.
  - Colonnes **numériques** clés : délais, quantités, prix, montants, profits, IDs.
  - Dates (`order date (DateOrders)`, `shipping date (DateOrders)`) sont pour l’instant au format texte (object).

**Idée importante** : on a un gros fichier transactionnel riche, avec un grain ligne de commande, parfait pour une future table de faits.


In [2]:
import pandas as pd

path = "DataCoSupplyChainDataset.csv"
df = pd.read_csv(path, encoding="latin1")

df.shape

(180519, 53)

In [3]:
df.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

In [4]:
df.head(3)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class


In [5]:
df.dtypes

Type                              object
Days for shipping (real)           int64
Days for shipment (scheduled)      int64
Benefit per order                float64
Sales per customer               float64
Delivery Status                   object
Late_delivery_risk                 int64
Category Id                        int64
Category Name                     object
Customer City                     object
Customer Country                  object
Customer Email                    object
Customer Fname                    object
Customer Id                        int64
Customer Lname                    object
Customer Password                 object
Customer Segment                  object
Customer State                    object
Customer Street                   object
Customer Zipcode                 float64
Department Id                      int64
Department Name                   object
Latitude                         float64
Longitude                        float64
Market          

### 2. Valeurs manquantes
- `Product Description` : **100 % manquante** → inutilisable pour la V1, on la met de côté.
- `Order Zipcode` : très forte proportion de valeurs manquantes (~86 %) → à exclure ou à traiter plus tard.
- `Customer Lname` : 8 valeurs manquantes → négligeable.
- `Customer Zipcode` : 3 valeurs manquantes → négligeable.
- Les colonnes importantes pour l’analyse (délais, ventes, profits, statut de livraison, marché, etc.) n’ont **pas de NaN**.

**Idée importante** : la qualité est globalement bonne sur les variables business/logistique, seuls quelques champs sont trop vides pour être utiles.

In [6]:
missing = df.isna().sum().sort_values(ascending=False)
missing.head(15)

Product Description              180519
Order Zipcode                    155679
Customer Lname                        8
Customer Zipcode                      3
Days for shipment (scheduled)         0
Sales per customer                    0
Benefit per order                     0
Delivery Status                       0
Late_delivery_risk                    0
Customer City                         0
Customer Country                      0
Category Id                           0
Category Name                         0
Customer Fname                        0
Customer Email                        0
dtype: int64

### 3. Cardinalité (valeurs uniques)

- `Order Item Id` : **180 519** valeurs uniques → grain naturel de la table de faits (1 ligne = 1 item de commande).
- `Order Id` : **65 752** → une commande peut avoir plusieurs items.
- `Customer Id` / `Order Customer Id` : **20 652** clients.
- `order date (DateOrders)` : **65 752** dates distinctes → base pour une dimension temps.
- Localisation très fine :
  - `Order City` ≈ **3 600** villes,
  - `Latitude` ≈ **11 000**, `Longitude` ≈ **4 500**.

**Idée importante** :  
- Fait central = `Order Item Id`.  
- Dimensions naturelles = client, produit, temps, localisation, shipping.


In [7]:
uniques = df.nunique().sort_values(ascending=False)
uniques.head(15)

Order Item Id                 180519
order date (DateOrders)        65752
Order Id                       65752
shipping date (DateOrders)     63701
Benefit per order              21998
Order Profit Per Order         21998
Customer Id                    20652
Order Customer Id              20652
Latitude                       11250
Customer Street                 7458
Longitude                       4487
Order City                      3597
Order Item Total                2927
Sales per customer              2927
Customer Lname                  1109
dtype: int64

### 4. Mesures clés (stats rapides)

- Délais :
  - `Days for shipping (real)` : moyenne ≈ **3,5 jours**, min 0, max 6.
  - `Days for shipment (scheduled)` : moyenne ≈ **2,9 jours**, max 4.
- Ventes & profits :
  - `Sales` : moyenne ≈ **203,8**, min 9,99, max ≈ 1 999,99.
  - `Order Item Total` : moyenne ≈ **183,1**.
  - `Benefit per order` / `Order Profit Per Order` : moyenne ≈ **22**, min ≈ **−4 275**, max ≈ **912**.

**Idée importante** : on a de quoi analyser **rentabilité** (profits parfois négatifs) et **performance logistique** (délai réel vs prévu) de façon fine.


In [8]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Days for shipping (real),180519.0,3.497654,1.623722,0.000000,2.000000,3.000000,5.000000,6.000000
Days for shipment (scheduled),180519.0,2.931847,1.374449,0.000000,2.000000,4.000000,4.000000,4.000000
Benefit per order,180519.0,21.974989,104.433526,-4274.979980,7.000000,31.520000,64.800003,911.799988
Sales per customer,180519.0,183.107609,120.043670,7.490000,104.379997,163.990005,247.399994,1939.989990
Late_delivery_risk,180519.0,0.548291,0.497664,0.000000,0.000000,1.000000,1.000000,1.000000
Category Id,180519.0,31.851451,15.640064,2.000000,18.000000,29.000000,45.000000,76.000000
Customer Id,180519.0,6691.379495,4162.918106,1.000000,3258.500000,6457.000000,9779.000000,20757.000000
Customer Zipcode,180516.0,35921.126914,37542.461122,603.000000,725.000000,19380.000000,78207.000000,99205.000000
Department Id,180519.0,5.443460,1.629246,2.000000,4.000000,5.000000,7.000000,12.000000
Latitude,180519.0,29.719955,9.813646,-33.937553,18.265432,33.144863,39.279617,48.781933


In [9]:
df[["Days for shipping (real)", "Days for shipment (scheduled)"]].describe().T

,count,mean,std,min,25%,50%,75%,max
Days for shipping (real),180519.0,3.497654,1.623722,0.0,2.0,3.0,5.0,6.0
Days for shipment (scheduled),180519.0,2.931847,1.374449,0.0,2.0,4.0,4.0,4.0


In [10]:
df[["Sales", "Benefit per order", "Order Item Product Price", "Order Item Quantity"]].describe().T

,count,mean,std,min,25%,50%,75%,max
Sales,180519.0,203.772096,132.273077,9.99000,119.980003,199.919998,299.950012,1999.989990
Benefit per order,180519.0,21.974989,104.433526,-4274.97998,7.000000,31.520000,64.800003,911.799988
Order Item Product Price,180519.0,141.232550,139.732492,9.99000,50.000000,59.990002,199.990005,1999.989990
Order Item Quantity,180519.0,2.127638,1.453451,1.00000,1.000000,1.000000,3.000000,5.000000


### 5. Catégories logistiques & marché

- `Delivery Status` :
  - Late delivery : 98 977
  - Shipping on time : 32 196
  - Advance shipping : 41 592
  - Shipping canceled : 7 754

- `Shipping Mode` :
  - Standard Class : 107 752
  - Second Class : 35 216
  - First Class : 27 814
  - Same Day : 9 737

- `Market` :
  - LATAM : 51 594
  - Europe : 50 252
  - Pacific Asia : 41 260
  - USCA : 25 799
  - Africa : 11 614

**Idée importante** :  
- Gros volume de **retards de livraison** → KPI majeur.  
- Modes d’expédition et marchés bien structurés → parfaits pour des dimensions simples.


In [11]:
cat_cols = df.select_dtypes(include=['object']).columns

for col in ["Delivery Status", "Shipping Mode", "Market", "Order Status", "Customer Country", "Order Region"]:
    if col in cat_cols:
        print(f"\n Value counts for column: {col}")
        print(df[col].value_counts().head(10))


 Value counts for column: Delivery Status
Delivery Status
Late delivery        98977
Advance shipping     41592
Shipping on time     32196
Shipping canceled     7754
Name: count, dtype: int64

 Value counts for column: Shipping Mode
Shipping Mode
Standard Class    107752
Second Class       35216
First Class        27814
Same Day            9737
Name: count, dtype: int64

 Value counts for column: Market
Market
LATAM           51594
Europe          50252
Pacific Asia    41260
USCA            25799
Africa          11614
Name: count, dtype: int64

 Value counts for column: Order Status
Order Status
COMPLETE           59491
PENDING_PAYMENT    39832
PROCESSING         21902
PENDING            20227
CLOSED             19616
ON_HOLD             9804
SUSPECTED_FRAUD     4062
CANCELED            3692
PAYMENT_REVIEW      1893
Name: count, dtype: int64

 Value counts for column: Customer Country
Customer Country
EE. UU.        111146
Puerto Rico     69373
Name: count, dtype: int64

 Value counts

### 6. Conclusion Étape 1 (pour l’ETL)

- Grain choisi pour la table de faits : **Order Item Id**.
- Dimensions candidates :
  - Client (Customer Id…)
  - Produit (Product Card Id, Category, Department…)
  - Temps (à partir des dates de commande / shipping)
  - Localisation (pays, ville, région, market, lat/long)
  - Shipping (mode, statut, risque de retard)

- Colonnes à ignorer dès maintenant :
  - `Product Description` (100 % NaN)
  - `Order Zipcode` (trop incomplet pour la V1)


## Étape 2 – Mapping & préparation des dimensions

**Objectif**  
Passer du CSV brut à des structures claires pour le Data Warehouse :  
- définir les dimensions (client, produit, temps, localisation, shipping)  
- préparer les DataFrames dim_ à partir de `DataCoSupplyChainDataset.csv`. [web:88][web:98]


### 2.1 Staging DataFrame

On nettoie minimalement le CSV pour le rendre exploitable par l’ETL.

In [12]:
df_raw=df.copy()

cols_to_drope=[
    "Product Description",
    "Order Zipcode",
    "Customer Email",
    "Customer Password"
]

df_stg=df_raw.drop(columns=cols_to_drope, errors='ignore')

df_stg["order_date"] = pd.to_datetime(
    df_stg["order date (DateOrders)"],
    errors="coerce",
    infer_datetime_format=True
)
df_stg["shipping_date"] = pd.to_datetime(
    df_stg["shipping date (DateOrders)"],
    errors="coerce",
    infer_datetime_format=True
)

C:\Users\HP\AppData\Local\Temp\ipykernel_13756\987208982.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_stg["order_date"] = pd.to_datetime(
C:\Users\HP\AppData\Local\Temp\ipykernel_13756\987208982.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_stg["shipping_date"] = pd.to_datetime(


- `df_stg` = staging propre (180 519 lignes, 49 colonnes environ).  
- Dates prêtes pour construire une dimension temps et calculer les délais.


### 2.2 Dimension client (dim_customer)

On crée une table client unique avec id, nom, segment et localisation.

In [13]:
dim_customer = (
    df_stg[[
        "Customer Id",
        "Customer Fname",
        "Customer Lname",
        "Customer Segment",
        "Customer Country",
        "Customer City",
        "Customer State",
        "Customer Street",
        "Customer Zipcode",
    ]]
    .drop_duplicates(subset=["Customer Id"])
    .rename(columns={
        "Customer Id": "customer_id",
        "Customer Fname": "fname",
        "Customer Lname": "lname",
        "Customer Segment": "segment",
        "Customer Country": "country",
        "Customer City": "city",
        "Customer State": "state",
        "Customer Street": "street",
        "Customer Zipcode": "zipcode",
    })
    .reset_index(drop=True)
)

dim_customer.head()


,customer_id,fname,lname,segment,country,city,state,street,zipcode
0,20755,Cally,Holloway,Consumer,Puerto Rico,Caguas,PR,5365 Noble Nectar Island,725.0
1,19492,Irene,Luna,Consumer,Puerto Rico,Caguas,PR,2679 Rustic Loop,725.0
2,19491,Gillian,Maldonado,Consumer,EE. UU.,San Jose,CA,8510 Round Bear Gate,95125.0
3,19490,Tana,Tate,Home Office,EE. UU.,Los Angeles,CA,3200 Amber Bend,90027.0
4,19489,Orli,Hendricks,Corporate,Puerto Rico,Caguas,PR,8671 Iron Anchor Corners,725.0


- Une ligne par client (`customer_id`).  
- Attributs disponibles : nom, segment, pays, ville… → prêts pour les analyses “par client / segment / pays”.

### 2.3 Dimension produit (dim_product)

On crée une table produit unique avec catégories et prix de base.

In [14]:
dim_product = (
    df_stg[[
        "Product Card Id",
        "Product Name",
        "Category Id",
        "Category Name",
        "Department Id",
        "Department Name",
        "Product Category Id",
        "Product Price",
    ]]
    .drop_duplicates(subset=["Product Card Id"])
    .rename(columns={
        "Product Card Id": "product_id",
        "Product Name": "product_name",
        "Category Id": "category_id",
        "Category Name": "category_name",
        "Department Id": "department_id",
        "Department Name": "department_name",
        "Product Category Id": "product_category_id",
        "Product Price": "base_price",
    })
    .reset_index(drop=True)
)

dim_product.head()

,product_id,product_name,category_id,category_name,department_id,department_name,product_category_id,base_price
0,1360,Smart watch,73,Sporting Goods,2,Fitness,73,327.750000
1,365,Perfect Fitness Perfect Rip Deck,17,Cleats,4,Apparel,17,59.990002
2,627,Under Armour Girls' Toddler Spine Surge Runni,29,Shop By Sport,5,Golf,29,39.990002
3,502,Nike Men's Dri-FIT Victory Golf Polo,24,Women's Apparel,5,Golf,24,50.000000
4,278,Under Armour Men's Compression EV SL Slide,13,Electronics,3,Footwear,13,44.990002


- Une ligne par produit (`product_id`).  
- Attributs : nom, catégorie, département, prix → nécessaires pour les analyses par produit/catégorie.

### 2.4 Dimension temps (dim_time)

On dérive une dimension temps à partir de la date de commande.

In [15]:
time_order = (
    df_stg[["order_date"]]
    .dropna()
    .drop_duplicates()
    .sort_values("order_date")
)

time_order["date_id"]=time_order["order_date"].dt.strftime("%Y%m%d").astype(int)
time_order["year"] = time_order["order_date"].dt.year
time_order["month"] = time_order["order_date"].dt.month
time_order["day"] = time_order["order_date"].dt.day

dim_time = time_order[["date_id", "order_date", "year", "month", "day"]].reset_index(drop=True) 

dim_time.head()

,date_id,order_date,year,month,day
0,20150101,2015-01-01 00:00:00,2015,1,1
1,20150101,2015-01-01 00:21:00,2015,1,1
2,20150101,2015-01-01 01:03:00,2015,1,1
3,20150101,2015-01-01 01:24:00,2015,1,1
4,20150101,2015-01-01 02:06:00,2015,1,1


- Une ligne par jour de commande (`date_id`).  
- Attributs : année, mois, jour → pour l’analyse des tendances et saisonnalité.

### 2.5 Dimension localisation (dim_location)

On crée une dimension pour pays, villes, régions et marchés.

In [16]:
dim_location = (
    df_stg[[
        "Customer Country",
        "Customer City",
        "Order City",
        "Order Region",
        "Market",
        "Latitude",
        "Longitude",
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_location["loction_id"]=dim_location.index + 1
dim_location.head()

,Customer Country,Customer City,Order City,Order Region,Market,Latitude,Longitude,loction_id
0,Puerto Rico,Caguas,Bekasi,Southeast Asia,Pacific Asia,18.251453,-66.037056,1
1,Puerto Rico,Caguas,Bikaner,South Asia,Pacific Asia,18.279451,-66.037064,2
2,EE. UU.,San Jose,Bikaner,South Asia,Pacific Asia,37.292233,-121.881279,3
3,EE. UU.,Los Angeles,Townsville,Oceania,Pacific Asia,34.125946,-118.291016,4
4,Puerto Rico,Caguas,Townsville,Oceania,Pacific Asia,18.253769,-66.037048,5


- Une ligne par combinaison de localisation.  
- `location_id` servira de clé vers la fact table pour analyser par pays, région, marché.

### 2.6 Dimension shipping (dim_shipping)

On encode les modes d’expédition et statuts de livraison.

In [17]:
dim_shipping = (
    df_stg[[
        "Shipping Mode",
        "Delivery Status",
        "Late_delivery_risk",
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_shipping["shipping_id"] = dim_shipping.index + 1
dim_shipping.head()

,Shipping Mode,Delivery Status,Late_delivery_risk,shipping_id
0,Standard Class,Advance shipping,0,1
1,Standard Class,Late delivery,1,2
2,Standard Class,Shipping on time,0,3
3,Standard Class,Shipping canceled,0,4
4,First Class,Late delivery,1,5


- Une ligne par combinaison mode/statut/risque.  
- `shipping_id` permettra d’analyser retards par mode, statut, etc.

## Conclusion Étape 2

- Étape 2 = mapping logique → DataFrames de **dimensions** créés :  
  - `dim_customer`, `dim_product`, `dim_time`, `dim_location`, `dim_shipping`.  
- On a maintenant le “contexte” pour analyser les faits.  
- Prochaine étape (Étape 3) : construire la **fact table** (fact_orders) avec le bon grain (Order Item) et les mesures (quantités, ventes, profits, délais).

## Étape 3 – Construction de la table de faits (fact_orders)

**Objectif**  
Construire `fact_orders` au grain **ligne de commande** (Order Item), avec :  
- les **clés** nécessaires vers les dimensions,  
- les **mesures** (quantité, ventes, remises, profits, délais). [web:88][web:107]

### 3.1 Base de fact_orders

On filtre et renomme les colonnes de `df_stg` pour la table de faits.

In [21]:
fact_orders = df_stg[[
    "Order Item Id",
    "Order Id",
    "Order Customer Id",
    "Product Card Id",
    "order_date",
    "shipping_date",
    "Order City",
    "Order Region",
    "Market",
    "Shipping Mode",
    "Delivery Status",
    "Late_delivery_risk",
    "Order Item Quantity",
    "Order Item Product Price",
    "Order Item Discount",
    "Order Item Discount Rate",
    "Order Item Total",
    "Sales",
    "Benefit per order",
    "Order Profit Per Order",
    "Days for shipping (real)",
    "Days for shipment (scheduled)",
]].rename(columns={
    "Order Item Id": "order_item_id",
    "Order Id": "order_id",
    "Order Customer Id": "customer_id",
    "Product Card Id": "product_id",
    "Order City": "order_city",
    "Order Region": "order_region",
    "Order Item Quantity": "quantity",
    "Order Item Product Price": "unit_price",
    "Order Item Discount": "discount_amount",
    "Order Item Discount Rate": "discount_rate",
    "Order Item Total": "line_total",
    "Benefit per order": "order_benefit",
    "Order Profit Per Order": "order_profit",
    "Days for shipping (real)": "days_shipping_real",
    "Days for shipment (scheduled)": "days_shipping_sched",
})

fact_orders.head()

,order_item_id,order_id,customer_id,product_id,order_date,shipping_date,order_city,order_region,Market,Shipping Mode,...,quantity,unit_price,discount_amount,discount_rate,line_total,Sales,order_benefit,order_profit,days_shipping_real,days_shipping_sched
0,180517,77202,20755,1360,2018-01-31 22:56:00,2018-02-03 22:56:00,Bekasi,Southeast Asia,Pacific Asia,Standard Class,...,1,327.75,13.110000,0.04,314.640015,327.75,91.250000,91.250000,3,4
1,179254,75939,19492,1360,2018-01-13 12:27:00,2018-01-18 12:27:00,Bikaner,South Asia,Pacific Asia,Standard Class,...,1,327.75,16.389999,0.05,311.359985,327.75,-249.089996,-249.089996,5,4
2,179253,75938,19491,1360,2018-01-13 12:06:00,2018-01-17 12:06:00,Bikaner,South Asia,Pacific Asia,Standard Class,...,1,327.75,18.030001,0.06,309.720001,327.75,-247.779999,-247.779999,4,4
3,179252,75937,19490,1360,2018-01-13 11:45:00,2018-01-16 11:45:00,Townsville,Oceania,Pacific Asia,Standard Class,...,1,327.75,22.940001,0.07,304.809998,327.75,22.860001,22.860001,3,4
4,179251,75936,19489,1360,2018-01-13 11:24:00,2018-01-15 11:24:00,Townsville,Oceania,Pacific Asia,Standard Class,...,1,327.75,29.500000,0.09,298.250000,327.75,134.210007,134.210007,2,4


- `fact_orders` contient une ligne par `order_item_id`.  
- Toutes les mesures importantes sont présentes (quantité, prix, total, ventes, profits, délais).

### 3.2 Mesures dérivées

On ajoute des colonnes calculées pour la logistique et la rentabilité.

In [22]:
#Retard qui peut être négatif si livré en avance
fact_orders["delay_days"] =(
    fact_orders["days_shipping_real"] - 
    fact_orders["days_shipping_sched"])

#Marge = Proft/Ventes, en évitant la division par zéro
fact_orders["margin_ratio"] = fact_orders.apply(
    lambda row: row["order_profit"] / row["line_total"] 
    if row["line_total"] not in (0,None) 
    else None,
    axis=1
)

fact_orders[[
    "order_item_id",
    "days_shipping_real",
    "days_shipping_sched",
    "delay_days",
    "Sales",
    "order_profit",
    "margin_ratio",
]].head()

,order_item_id,days_shipping_real,days_shipping_sched,delay_days,Sales,order_profit,margin_ratio
0,180517,3,4,-1,327.75,91.250000,0.290014
1,179254,5,4,1,327.75,-249.089996,-0.800006
2,179253,4,4,0,327.75,-247.779999,-0.800013
3,179252,3,4,-1,327.75,22.860001,0.074998
4,179251,2,4,-2,327.75,134.210007,0.449992


- `delay_days` prêt pour analyser les retards moyens/par mode/par marché.  
- `margin_ratio` exploitable pour comparer la rentabilité des produits/segments.

### 3.3 Clés de liaison

On vérifie que les colonnes de fact_orders existent bien dans les dimensions.

In [24]:
#just quick checks 
fact_orders["customer_id"].isin(dim_customer["customer_id"]).all()
fact_orders["product_id"].isin(dim_product["product_id"]).all()

np.True_

In [25]:
fact_orders=fact_orders.merge(
    dim_time[["date_id", "order_date"]],
    on="order_date",
    how="left"
)

fact_orders[["order_item_id","order_date","date_id"]].head()

,order_item_id,order_date,date_id
0,180517,2018-01-31 22:56:00,20180131
1,179254,2018-01-13 12:27:00,20180113
2,179253,2018-01-13 12:06:00,20180113
3,179252,2018-01-13 11:45:00,20180113
4,179251,2018-01-13 11:24:00,20180113


- fact_orders est alignée avec les dimensions via les clés naturelles.  
- `date_id` est prêt si on veut un vrai lien fact_orders → dim_time.

## Conclusion Étape 3

- `fact_orders` est construite au grain **Order Item** avec :
  - clés : order_item_id, order_id, customer_id, product_id, date_id, infos location & shipping.
  - mesures : quantity, unit_price, line_total, sales, order_profit, delays, margin_ratio.
- On a maintenant : dim_* + fact_orders → **schéma en étoile logique complet**.

## Étape 4 – Vérifications fact ↔ dimensions

**Objectif**  
Vérifier que fact_orders est cohérente avec les dimensions (pas de clés orphelines, bonnes tailles).

In [26]:
df_stg.shape, fact_orders.shape

((180519, 51), (180519, 25))

- Même nombre de lignes !

In [27]:
#Tous les clients de la fact existent dans la dim_customer?
ok_customer = fact_orders["customer_id"].isin(dim_customer["customer_id"]).all()

#Toutes les produits de la fact existent dans la dim_product?
ok_product = fact_orders["product_id"].isin(dim_product["product_id"]).all()

ok_customer, ok_product

(np.True_, np.True_)

- `ok_customer` et `ok_product` sont True → pas de clés client/produit orphelines.

In [28]:
#Dim_temps !
fact_orders["date_id"].isna().mean()

np.float64(0.0)

## Étape 5 – Préparer l’export vers le Data Warehouse

**Objectif**  
Rendre les tables prêtes à être chargées dans un SGBD (ou utilisées par Power BI).

In [29]:
dim_customer.to_csv("dim_customer.csv", index=False)
dim_product.to_csv("dim_product.csv", index=False)
dim_time.to_csv("dim_time.csv", index=False)
dim_location.to_csv("dim_location.csv", index=False)
dim_shipping.to_csv("dim_shipping.csv", index=False)
fact_orders.to_csv("fact_orders.csv", index=False)

- Tous les fichiers nécessaires pour construire le Data Warehouse sont prêts.  
- Ils pourront être chargés dans PostgreSQL ou directement utilisés par Power BI.

## Conclusion 
- Profiling terminé (Étape 1).  
- Staging + dimensions construites (Étape 2).  
- Table de faits construite avec mesures dérivées (Étape 3).  
- Cohérence fact ↔ dimensions vérifiée (Étape 4).  
- Exports CSV prêts pour le Data Warehouse / Power BI (Étape 5).
